<a href="https://colab.research.google.com/github/RayJi01/CuringBot/blob/main/Project_curingbot_dataprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
def preprocess_intents_json(intents_file):
    with open(intents_file, "r") as f:
        data = json.load(f)

    preprocessed_data = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            preprocessed_data.append(f"User: {pattern}\n")
            for response in intent["responses"]:
                preprocessed_data.append(f"Assistant: {response}\n")

    txt="".join(preprocessed_data)
    # Space out the punctuations
    txt=txt.replace(',',' , ').replace('.',' . ').replace('?',' ? ').replace('!',' ! ')
    txt=txt.replace('  ',' ')
    # Add <s> to beginning of text
    txt=txt.replace("User: ",'User: <s> ')
    txt=txt.replace("Assistant: ",'Assistant: <s> ')
    # Add <s> and </s> to every middle sentences
    txt=txt.replace("! ",'! </s> <s> ')
    txt=txt.replace("...... ",'...... </s> <s> ')
    txt=txt.replace(". ",'. </s> <s> ')
    txt=txt.replace("? ",'? </s> <s> ')
    txt=txt.replace(" <s> \n",'\n')
    return txt


def save_preprocessed_data(preprocessed_data, output_file):
    with open(output_file, "w") as f:
        f.write(preprocessed_data)


intents_file = "intents.json"
output_file = "mental_health_data.txt"


preprocessed_data = preprocess_intents_json(intents_file)
save_preprocessed_data(preprocessed_data, output_file)

FileNotFoundError: [Errno 2] No such file or directory: 'intents.json'

In [ ]:
preprocessed_data_csv=preprocess_csv("train-2.csv")
save_preprocessed_data(preprocessed_data_csv, "dataset3-3512.txt")

IndexError: list index out of range

In [ ]:
import csv
import os
def preprocess_csv(csv_file):
  preprocessed_data_csv = []
  count=-1
  with open(csv_file, mode ='r')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      count+=1
      if count<1:
        continue
      lines[0]=lines[0].replace("   "," ").replace("\n","")
      preprocessed_data_csv.append(f"User: {lines[0]}\n")
      preprocessed_data_csv.append(f"Assistant: {lines[1]}\n")
  print(f"The document contains {count} entries")
  txt="".join(preprocessed_data_csv)
  # Space out the punctuations
  txt=txt.replace(',',' , ').replace('.',' . ').replace('?',' ? ').replace('!',' ! ')
  txt=txt.replace('  ',' ')
  # Add <s> to beginning of text
  txt=txt.replace("User: ",'User: <s> ')
  txt=txt.replace("Assistant: ",'Assistant: <s> ')
  # Add <s> and </s> to every middle sentences
  txt=txt.replace("! ",'! </s> <s> ')
  txt=txt.replace("...... ",'...... </s> <s> ')
  txt=txt.replace(". ",'. </s> <s> ')
  txt=txt.replace("? ",'? </s> <s> ')
  txt=txt.replace(" <s> \n",'\n')
  return txt

# os.remove("dataset2-3512.txt")
preprocessed_data_csv=preprocess_csv("train.csv")
save_preprocessed_data(preprocessed_data_csv, "dataset2-3512.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
import re
preprocessed_dataset_3=""
faulty_entries=[]
quote_pattern = r'"([^"]*\s+[^"]*)"'

# Replacement function to strip off quotation marks
def remove_quotes(match):
    return match.group(1)

with open("train-2.csv", mode ='r')as file:
    csvFile = csv.reader(file)
    count=0
    for lines in csvFile:
      if lines[0]=="conversations":
        continue
      # Change all value value parentheses to single quote to avoid escaping
      txt=lines[0].replace("'value\': \"",'\'value\': \'')
      txt=txt.replace("\"}",'\'}')
      # Remove all quotes that are not topped on single words
      txt = re.sub(quote_pattern, remove_quotes, txt)
      # Change all double quotes to escaped double quotes
      txt=txt.replace("\"",'\\"')
      # Change all escaped single quotes back to single quotes
      txt=txt.replace("\\'","'")
      # Change all designated places' single quotes to double quots
      txt=txt.replace("'from'",'"from"')
      txt=txt.replace("'value'",'"value"')
      txt=txt.replace("\"value\": '",'"value": "')
      txt=txt.replace("'}",'"}')
      txt=txt.replace("'human'",'"human"')
      txt=txt.replace("'gpt'",'"gpt"')
      # Space out the punctuations
      txt=txt.replace(',',' , ').replace('.',' . ').replace('?',' ? ').replace('!',' ! ')
      txt=txt.replace('  ',' ')
      # Add <s> to beginning of text
      txt=txt.replace("\"value\": \"",'"value": "<s> ')
      # Add <s> and </s> to every middle sentences
      txt=txt.replace("! ",'! </s> <s> ')
      txt=txt.replace("...... ",'...... </s> <s> ')
      txt=txt.replace(". ",'. </s> <s> ')
      txt=txt.replace("? ",'? </s> <s> ')
      txt=txt.replace(" <s> \"}",'"}')
      txt=txt[1:-1].split("\n ")
      for entry in txt:
        if entry[-1]!='}':
          entry+='}'
        if entry[-2]!='"':
          entry=entry[0:-1]+'"}'
        try:
          entry_dict=json.loads(entry) # Remove all quotation marks as they can be difficult to tokenize
          count+=1
          if entry_dict['from']=="human":
            preprocessed_dataset_3+=(f"User: {entry_dict['value']}\n")
          else:
            preprocessed_dataset_3+=(f"Assistant: {entry_dict['value']}\n")
        except:
          faulty_entries.append(entry)

save_preprocessed_data(preprocessed_dataset_3, f"dataset3-{count}.txt")


In [ ]:
txt=line[0].replace("\'from\'",'"from"')
txt=txt.replace("\'value\'",'"value"')
txt=txt.replace("\'human\'",'"human"')
txt=txt.replace("\'gpt\'",'"gpt"')
txt=txt[1:-2].split("\n ")
json.loads(txt[0])
# txt[0]

{'from': 'human',
 'value': "I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me."}

In [ ]:
print(len(faulty_entries))

0


In [ ]:
print(count)

840399


In [ ]:
line

['[{\'from\': \'human\', \'value\': "I\'ve been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me."}\n {\'from\': \'gpt\', \'value\': "Hey there, I\'m here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what\'s been going on?"}\n {\'from\': \'human\', \'value\': "I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. It\'s been a really moving experience for me."}\n {\'from\': \'gpt\', \'value\': "I can understand how it can be overwhelming when we\'re faced with higher expectations. It\'s okay to acknowledge your emotions and allow yourself to feel sad in this situation. It\'s an important part of the healing process. What specific challenges have you been facing at work?"}\n {\'from\': \'human\', \'value\': "Well, the workload has increased significantly, and I find it hard to